# Usando o algoritmo de Viterbi para rotular sequências de POS tagging

Baseado no tutorial: https://www.kaggle.com/deepanshusinha/hmms-and-viterbi-algorithm-for-pos-tagging

In [1]:
#bibliotecas relevantes
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [2]:
# [('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj')]
# nltk.download()
# leitura de sentenças com as tags
data = list(nltk.corpus.floresta.tagged_sents())

In [3]:
print(data[0])

[('Um', '>N+art'), ('revivalismo', 'H+n'), ('refrescante', 'N<+adj')]


In [4]:
# as tags do corpus floresta consistem de alguma 
# informação sintática e mais a tag de POS. Vamos retirar 
# essa informação sintática e ficar só com as tags POS. 
# Fonte: http://www.nltk.org/howto/portuguese_en.html
def simplify_tag(t):
    if "+" in t:
        return t[t.index("+")+1:]
    else:
        return t
    
new_data = []
for sent in data:
    new_sent = []
    for (w, t) in sent:
        new_sent.append((w, simplify_tag(t)))
    new_data.append(new_sent)

In [6]:
# vendo como os nossos dados ficaram
print(new_data[0])

[('Um', 'art'), ('revivalismo', 'n'), ('refrescante', 'adj')]


In [7]:
# dividindo dados em treino e validação com proporção 95:5
random.seed(1234)
train_set, test_set = train_test_split(new_data, train_size=0.95, test_size=0.05)

print("Tamanho do conjunto de treino -", len(train_set))
print("Tamanho do conjunto de teste -", len(test_set))

Tamanho do conjunto de treino - 8802
Tamanho do conjunto de teste - 464


In [8]:
# Pegando a lista de treino e teste das palavras rotuladas
train_tagged_words = [tup for sent in train_set for tup in sent]
print("Palavras de treino rotuladas - ", len(train_tagged_words))

test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print("Palavras de teste rotuladas - ", len(test_tagged_words))

Palavras de treino rotuladas -  201476
Palavras de teste rotuladas -  10376


In [9]:
# dando uma checada
print(train_tagged_words[:5])

[('O', 'art'), ('reajuste', 'n'), ('é', 'v-fin'), ('de', 'prp'), ('6,08', 'num')]


In [10]:
# computar a probabilidade e emissão para uma dada palavra 
# para uma dada tag
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(tag_list)    
    word_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]    
    word_given_tag_count = len(word_given_tag_list)    
    
    return (word_given_tag_count, tag_count)

In [11]:
# computer a as probabilidades de transição da tag anterior e da próxima tag
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags_list = [tag for tag in tags if tag == t1]
    t1_tags_count = len(t1_tags_list)
    
    t2_given_t1_list = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    t2_given_t1_count = len(t2_given_t1_list)
    
    return(t2_given_t1_count, t1_tags_count)

In [12]:
# tokens no conjunto de treino
train_tagged_tokens = [tag[0] for tag in train_tagged_words]

# POS tags para os tokens no conjunto de treino
train_tagged_pos_tokens = [tag[1] for tag in train_tagged_words]

# construção do vocabulário de treino (conjunto único de palavras)
training_vocabulary_set = set(train_tagged_tokens)

# construção de POS tags em um conjunto (conjunto único de tags)
training_pos_tag_set = set(train_tagged_pos_tokens)

In [13]:
# computando P(w/t) e armazenando na matriz  [Tags x Vocabulario]. Essa é uma matriz com dimensão 
# de len(training_pos_tag_set) X len(training_vocabulary_set)

len_pos_tags = len(training_pos_tag_set)
len_vocab = len(training_vocabulary_set)

In [14]:
# criando uma matriz de transição tag x tag de training_pos_tag_set
# cada coluna é t2, cada linha é  t1
# assim M(i, j) representa P(tj dado ti)

tags_matrix = np.zeros((len_pos_tags, len_pos_tags), dtype='float32')
for i, t1 in enumerate(list(training_pos_tag_set)):
    for j, t2 in enumerate(list(training_pos_tag_set)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
# convertendo a matriz para um data frame para melhor legibilidade
tags_df = pd.DataFrame(tags_matrix, columns = list(training_pos_tag_set), index=list(training_pos_tag_set))

In [16]:
# Algoritmo de Viterbi simples
def Vanilla_Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # Essa eh a nossa função de score (psi)
            # computar a emissão e probabilidade de estados
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # pegando o estado para o qual a probabilidade eh maxima
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [17]:
test_tagged_words = ['o','livro','está','na','mesa']
tagged_seq = Vanilla_Viterbi(test_tagged_words)

In [18]:
print(tagged_seq)

[('o', 'art'), ('livro', 'n'), ('está', 'v-fin'), ('na', 'pron-pers'), ('mesa', 'n')]


In [22]:
test_tagged_words1 = ['o','rio','é','dele']
tagged_seq1 = Vanilla_Viterbi(test_tagged_words1)

In [23]:
print(tagged_seq1)

[('o', 'art'), ('rio', 'n'), ('é', 'v-fin'), ('dele', '2]+adv')]


In [24]:
test_tagged_words2 = ['eu','rio','dele']
tagged_seq2 = Vanilla_Viterbi(test_tagged_words2)

In [25]:
print(tagged_seq2)

[('eu', 'pron-pers'), ('rio', 'n'), ('dele', '2]+adv')]
